<a href="https://colab.research.google.com/github/sidharthofficial/BERT/blob/main/DistilBERT_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Reference from: https://huggingface.co/docs/transformers/training

In [2]:
from google.colab import files
uploaded = files.upload()

Saving SMSSpamCollection.txt to SMSSpamCollection (1).txt


In [3]:
import pandas as pd
df = messages = pd.read_csv('/content/SMSSpamCollection.txt', sep='\t', names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
df.shape

(5572, 2)

In [5]:
X=list(df['message'])
y=list(df['label'])


In [6]:
X[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [7]:
y[0]

'ham'

In [8]:
# 'spam' is set to True

y=list(pd.get_dummies(y,drop_first=True)['spam'])


In [9]:
y[0]

False

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [11]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [13]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [14]:
!pip install transformers # Ensure TensorFlow dependencies are installed


In [15]:
from transformers import TFDistilBertForSequenceClassification, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


In [16]:
model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=training_args.learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ['accuracy']
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)


# Train the model
model.fit(
    train_dataset.shuffle(1000).batch(training_args.per_device_train_batch_size),
    epochs=training_args.num_train_epochs,
    batch_size=training_args.per_device_train_batch_size,
    validation_data=test_dataset.batch(training_args.per_device_eval_batch_size),
)
"""
# Save the model (optional)
model.save_pretrained("./results")
"""

# Save the model (optional)
model.save_pretrained("./results")




Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/2
558/558 [==============================] - 175s 236ms/step - loss: 0.0723 - accuracy: 0.9825 - val_loss: 0.0283 - val_accuracy: 0.9937
Epoch 2/2
558/558 [==============================] - 120s 214ms/step - loss: 0.0322 - accuracy: 0.9910 - val_loss: 0.0279 - val_accuracy: 0.9955


In [17]:
"""
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()

model.evaluate(test_dataset)
"""

# Evaluate the model
results = model.evaluate(
    test_dataset.batch(training_args.per_device_eval_batch_size),
    return_dict=True  # To get results as a dictionary
)

70/70 [==============================] - 10s 136ms/step - loss: 0.0279 - accuracy: 0.9955


In [18]:
predictions = model.predict(test_dataset)

1115/1115 [==============================] - 15s 12ms/step


In [19]:
print(type(predictions))  # Check the type of the prediction output
print(len(predictions)) if isinstance(predictions, (list, tuple)) else print("Not a list or tuple")


<class 'transformers.modeling_tf_outputs.TFSequenceClassifierOutput'>
Not a list or tuple


In [20]:
from sklearn.metrics import confusion_matrix
import numpy as np

output = np.argmax(predictions.logits, axis=1)

cm=confusion_matrix(y_test,output)
cm

array([[954,   1],
       [  4, 156]])